In [1]:
import boto3

In [9]:
import os
from os.path import basename, dirname
def replicate_s3_objects(source, target, keep_structure='False', **kwargs):
    
    # source processing
    _, path = source.split(":", 1)
    path = path.lstrip("/")
    sourcebucket, sourceprefix = path.split("/", 1)
    
    s3 = boto3.resource('s3',
                    aws_access_key_id=kwargs['SOURCE_ACCESS_ID'],
                    aws_secret_access_key=kwargs['SOURCE_ACCESS_SECRET']
                    )
    
    fileobj = '/tmp/' +basename(source)
    print('Downloading file...',fileobj)
    
    done = s3.Object(sourcebucket, sourceprefix).download_file(fileobj)
        
    
    # destination processing
    _, path = target.rstrip('/').split(":", 1)
    path = path.lstrip("/")
    try:
        targetbucket, targetprefix = path.split("/", 1)
        if 'True' in keep_structure:
            targetprefix = targetprefix + '/' +sourceprefix
        else:
            targetprefix = targetprefix + '/' +basename(source)
    except ValueError:
        targetbucket = path.split("/", 1)[0]
        if 'True' in keep_structure:
            targetprefix = sourceprefix
        else:
            targetprefix = basename(source)
    
    
    print('Target :',targetbucket,targetprefix)
    s3 = boto3.resource('s3',
                    aws_access_key_id=kwargs['TARGET_ACCESS_ID'],
                    aws_secret_access_key=kwargs['TARGET_ACCESS_SECRET']
                    )
    
    
    try:
        s3.Object(targetbucket,targetprefix ).upload_file(fileobj,{'ServerSideEncryption': "AES256"})
        print('Removing the file')
        os.remove(fileobj)
    except Exception: 
        raise Exception('Upload failed')    

In [12]:
replicate_s3_objects('s3://xx/odtc1_obsmodel_fieldpred.csv',
                     's3://xx2/data/test/',
                     'False',**args)

('Downloading file...', '/tmp/odtc1_obsmodel_fieldpred.csv')
('Target :', 'globalit-np', 'projects/ASPIII/applications/INSEASON/data/test/odtc1_obsmodel_fieldpred.csv')
Removing the file
